In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials 
import pandas as pd
import json
import time
import numpy as np

In [2]:
client_id = "125fcec321e048db972933f9f364d74e"
client_secret = "615237e26e0c4d22a13889fd7b91a46a"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

In [3]:
names = "red velvet"
result = sp.search(names, type='artist')
artist = result['artists']['items'][0]
artist_id = artist['id']
artist_genre = artist['genres']
# ttl_albums = [sp.artist_albums(artist_id, album_type=album_type) for album_type in ['album', 'single']]

In [6]:
print(artist['popularity'])

72


In [3]:
# try hashtag
artist_names = [
 'hashtag' 
]

In [4]:
spotify_records = {
                'artist_name': [],
                'artist_genre': [],
                'followers': [],
                'popularity': []
}
for name in artist_names:
    result = sp.search(name, type='artist')  # Search for the artist

    if result['artists']['items']:
        spotify_records['artist_name'].append(name)
        
        artist = result['artists']['items'][0]
        artist_genre = artist['genres']
        artist_pop = artist['popularity']
        spotify_records['artist_genre'].append(artist_genre)
        spotify_records['popularity'].append(artist_pop)
        
        artist_id = artist['id']
        artist_info = sp.artist(artist_id)
        followers = artist_info['followers']['total']
        spotify_records['followers'].append(followers) 
               
    else:
        print(f"No artist found for {name}")

print(spotify_records)

{'artist_name': ['hashtag'], 'artist_genre': [['movie tunes', 'social media pop']], 'followers': [422003], 'popularity': [43]}


In [14]:
df_spotify_records = pd.DataFrame(spotify_records)
df_spotify_records

,artist_name,artist_genre,followers,popularity
0,the weeknd,"[canadian contemporary r&b, canadian pop, pop]",77988127,95
1,Ed Sheeran,"[pop, singer-songwriter pop, uk pop]",114665079,88
2,P!NK,"[dance pop, pop]",16229343,80
3,Morgan Wallen,[contemporary country],8245937,86
4,Harry Styles,[pop],30945602,84
5,Billy Joel,"[album rock, classic rock, heartland rock, mel...",5665011,75
6,Taylor Swift,[pop],99663761,100
7,Coldplay,"[permanent wave, pop]",48925591,87
8,Doja Cat,"[dance pop, pop]",28340610,88


In [15]:
df_spotify_records.to_csv('spotify_records2.csv', index=False)

In [ ]:
# getting more songs features

In [33]:
def get_artist_album_info(artist_names):
    
    album_info = {
                'artist_name': [],
                'artist_id': [],
                'artist_genre': [],
                'album_id': [],
                'album_name': [],
                'album_type': [],
                'album_ttl_tracks': [],
                'release_date': [],
                'track_name':[],
#                 'track_id':[],
                'track_duration_ms':[],
                'danceability':[],
                'energy':[],
                'key':[],
                'mode':[],
                'speechiness':[],
                'acousticness':[],
                'instrumentalness':[],
                'liveness':[],
                'valence':[],
                'tempo':[],
#                 'popularity':[],
                'track_count_check':[]    
            }
    
    for artist in artist_names:
        result = sp.search(artist, type='artist')
        if result:
            artist = result['artists']['items'][0]
            artist_name = artist['name']
            artist_id = artist['id']
            artist_genre = artist['genres']
            ttl_albums = [sp.artist_albums(artist_id, album_type=album_type) for album_type in ['album', 'single']]

            for album_set in ttl_albums:
                for album in album_set['items']:
                    album_id = album['id']
                    album_name = album['name']
                    album_type = album['album_type']
                    album_ttl_tracks = album['total_tracks']
                    release_date = album['release_date']
                    
                    #                 getting info for track:
                    tracks = sp.album_tracks(album_id)
                    items = tracks['items']
                    track_count_check = 0
                    for track in items:
                        track_name = track['name']
                        track_duration_ms = track['duration_ms']

                        track_ids = [track['id'] for track in items]
                        features_list = sp.audio_features(track_ids)

                        
#                        先append feature以上的东西
                        album_info['artist_name'].append(artist_name)
                        album_info['artist_id'].append(artist_id)
                        album_info['artist_genre'].append(artist_genre)
                        album_info['album_id'].append(album_id)
                        album_info['album_name'].append(album_name)
                        album_info['album_type'].append(album_type)
                        album_info['album_ttl_tracks'].append(album_ttl_tracks)
                        album_info['release_date'].append(release_date)

                        album_info['track_name'].append(track_name)
#                         album_info['track_id'].append(track_id)
                        album_info['track_duration_ms'].append(track_duration_ms)
                    
#                       getting features for each track:
                    for features in features_list:
                        danceability = features['danceability']
                        energy = features['energy']
                        key = features['key']
                        mode = features['mode']
                        speechiness = features['speechiness']
                        acousticness = features['acousticness']
                        instrumentalness = features['instrumentalness']
                        liveness = features['liveness']
                        valence = features['valence']
                        tempo = features['tempo'] 
                        track_count_check+=1
                         
                        # start appending track's feature
                        album_info['danceability'].append(danceability)
                        album_info['energy'].append(energy)
                        album_info['key'].append(key)
                        album_info['mode'].append(mode)
                        album_info['speechiness'].append(speechiness)
                        album_info['acousticness'].append(acousticness)
                        album_info['instrumentalness'].append(instrumentalness)
                        album_info['liveness'].append(liveness)
                        album_info['valence'].append(valence)
                        album_info['tempo'].append(tempo)
                        album_info['track_count_check'].append(track_count_check)
                        

                
        else:
            print(f"can not find {artist} on spotify")
    
    return album_info
            

In [30]:
artist_names = ["winner"]
print(get_artist_album_info(artist_names))


Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=03MTY1omA35sy8rWJEtGcq:
 Max Retries, reason: too many 429 error responses

In [60]:
data = get_artist_album_info(artist_names)
df = pd.DataFrame(data)
df

,artist_name,artist_id,artist_genre,album_id,album_name,album_type,album_ttl_tracks,release_date,track_name,track_duration_ms,...,energy,key,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_count_check
0,WINNER,5DuzBeOgFwViFcv00Q5PFb,"[k-pop, k-pop boy group]",4JL5JcIdo3gl3pT5rZZOeQ,Remember (JP EDITION),album,13,2020-07-22,REMEMBER - KR Ver.,224200,...,0.495,3,1,0.0316,0.7650,0.0,0.1070,0.251,130.102,1
1,WINNER,5DuzBeOgFwViFcv00Q5PFb,"[k-pop, k-pop boy group]",4JL5JcIdo3gl3pT5rZZOeQ,Remember (JP EDITION),album,13,2020-07-22,DDUK - JINU SOLO KR Ver.,210173,...,0.257,7,1,0.0382,0.8500,0.0,0.0948,0.357,137.736,2
2,WINNER,5DuzBeOgFwViFcv00Q5PFb,"[k-pop, k-pop boy group]",4JL5JcIdo3gl3pT5rZZOeQ,Remember (JP EDITION),album,13,2020-07-22,HOLD - KR Ver.,176346,...,0.656,8,1,0.0598,0.0475,0.0,0.0343,0.589,156.011,3
3,WINNER,5DuzBeOgFwViFcv00Q5PFb,"[k-pop, k-pop boy group]",4JL5JcIdo3gl3pT5rZZOeQ,Remember (JP EDITION),album,13,2020-07-22,JUST DANCE - KR Ver.,200866,...,0.595,0,1,0.0678,0.0619,0.0,0.1380,0.852,117.960,4
4,WINNER,5DuzBeOgFwViFcv00Q5PFb,"[k-pop, k-pop boy group]",4JL5JcIdo3gl3pT5rZZOeQ,Remember (JP EDITION),album,13,2020-07-22,MY BAD - KR Ver.,235320,...,0.520,6,0,0.0411,0.5280,0.0,0.1090,0.417,93.056,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,WINNER,5DuzBeOgFwViFcv00Q5PFb,"[k-pop, k-pop boy group]",7D2FML3EyRJcCoiEgUbRxT,EXIT : E,single,5,2016-02-01,SENTIMENTAL,209281,...,0.760,7,1,0.0619,0.2870,0.0,0.3770,0.712,118.060,2
287,WINNER,5DuzBeOgFwViFcv00Q5PFb,"[k-pop, k-pop boy group]",7D2FML3EyRJcCoiEgUbRxT,EXIT : E,single,5,2016-02-01,IMMATURE,192696,...,0.808,1,0,0.0750,0.0312,0.0,0.3000,0.736,126.044,3
288,WINNER,5DuzBeOgFwViFcv00Q5PFb,"[k-pop, k-pop boy group]",7D2FML3EyRJcCoiEgUbRxT,EXIT : E,single,5,2016-02-01,I'M YOUNG,213437,...,0.263,8,0,0.0399,0.8450,0.0,0.3650,0.258,129.614,4
289,WINNER,5DuzBeOgFwViFcv00Q5PFb,"[k-pop, k-pop boy group]",7D2FML3EyRJcCoiEgUbRxT,EXIT : E,single,5,2016-02-01,PRICKED (MINO&TAEHYUN),231793,...,0.428,9,0,0.0337,0.4740,0.0,0.3660,0.389,129.813,5


In [ ]:
# max retries reached

In [4]:
artist_names = [
"xikers"
    ,
"EVERGLOW",
"The Rose"
    ,
"ENHYPEN",
"tripleS",
"Stay-C",
"Radwimps",
"BABYMETAL, Metalocalypse: Dethklok",
"P1harmony",
"Loona",
"Twice",
"Jonathan Lee",
"SEVENTEEN",
"ATEEZ",
"TOMORROW X TOGETHER",
"Suga",
"Rain",
"KARD",
"NCT Dream",
"BLACKPINK",
"Red Velvet",
"Nmixx",
"Mayday",
"JJ Lin",
"Sabaton",
"Baek Z Young",
"Stray Kids",
"CIX",
"NCT 127",
"DPR Live, DPR IAN, DPR Cream",
"ITZY",
"MUSTB",
"(G)I-DLE",
"Monsta X",
"The Boyz",
"BTS",
"BABYMETAL",
"Day6",
"Got7",
"Little Simz",
"Mamamoo",
"Band Of Horses",
"Fei Yu Ching",
"Winner",
"Ssingssing",
"Jacky Cheung",
"Rene Liu",
"B.A.P.",
"Jane Zhang",
"Wanna One",
"Up10tion",
"Leehom Wang",
"Zhang Jun",
"Nine Percent",
"Chris Lee (Li Yuchun)",
"Yu-Ching Fei",
"Snh48",
"A-Mei",
"Kuang Program",
"FTIsland",
"Yu-Rong Yang",
"Lala Hsu",
"G-Dragon",
"Silence (Wang Su Long)",
"Bibi Zhou",
"Taeyang",
"Ronghao Li",
"Lion",
"Pu Shu",
"Tanya Chua",
"JJCC",
"Hebe Tien",
"Korn",
"Luo Tianyi",
"Joker Xue",
"Mua He Ruc Ro",
"Rica Matsumoto",
"Red Hot Chili Peppers",
"EXO",
"7 Senses",
"A Little Fever (ATF)",
"B1A4",
"Guns N' Roses",
"Akdong Musician",
"Momoiro Clover Z",
"CL",
"Bastille",
"Zhang Jie",
"Twins LOL",
"T-ara",
"Brilliance Yu",
"Sammi Cheng",
"Big Bang",
"Zhang Hang",
"Eir Aoi",
"Bigbang (Korea)",
"Yanzi Kepler",
"Lisa",
"TsuShiMaMiRe",
"Mua Thu Cho Em",
"2PM",
"Lee Juck",
"Mr.",
"Lady Gaga",
"C Allstar",
"Hebe Tien, Yoga Lin",
"RJD2",
"Vixx",
"Swervedriver",
"Tyme x Tujiko",
"2NE1",
"FTIsland, CNBlue",
"Lee Horn",
"2*Sweet",
"Inept",
"Set Your Goals",
"June",
"A Loss For Words",
"Lucky Boys Confusion",
"Chang Cheng-Yue",
"Chang Cheng-Yue, Free9",
"Box-O-Car",
"Danzig",
"Anthrax"
]

In [5]:
artist_data = get_artist_album_info(artist_names)

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=2XIAyrfqLiI5gWpZGRUtu9,2OdKV7e53hgK7Rr2ptQiww,6brb7SXjhqHOxsj0jkyQVH,3qiAJa0kAWCYigVU4oflbC,5Ae7aCUbM9MNmjj8pQbEVt,2MUwrlb8G1juxSFY05uSG5:
 Max Retries, reason: too many 429 error responses